In [84]:
import pandas as pd

In [85]:
# Load title.basics.tsv
title_basics = pd.read_csv('datas\\title.basics.tsv', sep='\t', low_memory=False)

In [ ]:
# Load movies.dat
movies = pd.read_csv('datas\\ml-1m\\movies.dat', sep='::', engine='python', encoding="latin-1", header=None, names=['MovieID', 'Title', 'Genres'])

# Load ratings.dat
ratings = pd.read_csv('datas\\ml-1m\\ratings.dat', sep='::', engine='python', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

# Load users.dat
users = pd.read_csv('datas\\ml-1m\\users.dat', sep='::', engine='python', header=None, names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])

In [ ]:
print(title_basics.isnull().sum())

tconst              0
titleType           0
primaryTitle       18
originalTitle      18
isAdult             0
startYear           0
endYear             0
runtimeMinutes      0
genres            561
dtype: int64


In [ ]:
print(movies.isnull().sum())

MovieID    0
Title      0
Genres     0
dtype: int64


In [ ]:
print(ratings.isnull().sum())

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64


In [ ]:
print(users.isnull().sum())

UserID        0
Gender        0
Age           0
Occupation    0
Zip-code      0
dtype: int64


In [ ]:
movies = movies.dropna()

# For ratings.dat - removing rows with any null values
ratings = ratings.dropna()

# For users.dat - removing rows with any null values
users = users.dropna()

# For title.basics.tsv from IMDb - removing rows with any null values
title_basics = title_basics.dropna()

In [ ]:
print(movies.columns)
print(ratings.columns)
print(users.columns)
print(title_basics.columns)

Index(['MovieID', 'Title', 'Genres'], dtype='object')
Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')
Index(['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], dtype='object')
Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')


In [ ]:
print(title_basics.head())

      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

  isAdult startYear endYear runtimeMinutes                    genres  
0       0      1894      \N              1         Documentary,Short  
1       0      1892      \N              5           Animation,Short  
2       0      1892      \N              5  Animation,Comedy,Romance  
3       0      1892      \N             12           Animation,Short  
4       0      1893      \N              1              Comedy,Short  


In [ ]:
print(users.head())

   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455


In [ ]:
print(movies.head())

   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy


In [ ]:
print(ratings.head())

   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


In [ ]:
movies['Year'] = movies['Title'].str.extract(r'\((\d{4})\)')
movies['Title'] = movies['Title'].str.replace(r'\s*\(\d{4}\)\s*', '', regex=True).str.strip()

In [ ]:
max_movie_id = movies['MovieID'].max()

title_basics['MovieID'] = range(max_movie_id + 1, max_movie_id + 1 + len(title_basics))

title_basics['startYear'] = title_basics['startYear'].astype(str)

title_basics['Year'] = title_basics['startYear'].str.extract(r'^(\d{4})$')
title_basics['Title'] = title_basics['primaryTitle'].str.strip()
title_basics['Genres'] = title_basics['genres'].str.split(',').str.join('|')

In [ ]:
merged_movies = pd.merge(movies, title_basics, on=['Title', 'Year', 'MovieID', 'Genres'], how='outer')
merged_movies = merged_movies.drop_duplicates(subset=['Title'])

KeyboardInterrupt: 

In [ ]:
merged_movies

,MovieID,Title,Genres,Year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,7548454,!,NaN,1998,tt31497793,tvEpisode,!,!,0,1998,\N,\N,"Animation,Comedy,Sci-Fi"
1,7917645,!! THIS VIDEO IS NOTHING BUT PAIN !! | Getting...,NaN,2018,tt32790928,tvEpisode,!! THIS VIDEO IS NOTHING BUT PAIN !! | Getting...,!! THIS VIDEO IS NOTHING BUT PAIN !! | Getting...,0,2018,\N,13,Comedy
2,7917452,!!POKING MY EYE OUT!! | Emily Wants to Play #2,NaN,2016,tt32788356,tvEpisode,!!POKING MY EYE OUT!! | Emily Wants to Play #2,!!POKING MY EYE OUT!! | Emily Wants to Play #2,0,2016,\N,9,Comedy
3,7532436,"!How to be a King, Princess or Superhero... in...",NaN,2020,tt31434107,tvEpisode,"!How to be a King, Princess or Superhero... in...","!How to be a King, Princess or Superhero... in...",0,2020,\N,\N,\N
4,6018412,!Next?,NaN,1994,tt2386381,tvSeries,!Next?,!Next?,0,1994,1995,\N,Documentary
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10907645,7884235,Кулон - The Pendant,NaN,2024,tt32654579,short,Кулон - The Pendant,Il ciondolo,0,2024,\N,\N,"Short,War"
10907646,6982888,Сall Me When You Get Home,NaN,2023,tt28970415,short,Сall Me When You Get Home,Steile otan ftaseis,0,2023,\N,6,"Drama,Short"
10907647,2234359,Самая любимая,NaN,2020,tt12283154,tvMovie,Самая любимая,Samaya luybimaya,0,2020,\N,102,Drama
10907648,5333613,【Oshi No Ko】,NaN,2023,tt21030032,tvSeries,【Oshi No Ko】,Oshi no Ko,0,2023,\N,24,"Animation,Drama,Fantasy"
